In [1]:
import openai

In [2]:
from openai import OpenAI

In [3]:
client = OpenAI()

In [4]:
import os

In [141]:
openai.api_key = os.getenv('OPENAI_TOKEN')

In [12]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [13]:
print(documents[0])

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.", 'section': 'General course-related questions', 'question': 'Course - When will the course start?', 'course': 'data-engineering-zoomcamp'}


In [21]:
from elasticsearch import Elasticsearch

In [22]:
es_client = Elasticsearch('http://localhost:9200')

In [24]:
es_client.info()

ObjectApiResponse({'name': '4b0bba479aad', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'BZbNoo7WQc--XPuyAyHCAg', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [25]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type":"text"},
            "section": {"type":"text"},
            "question": {"type":"text"},
            "course": {"type":"keyword"}
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [course-questions/3XyY7qX-Q1CqJy_yWvc79g] already exists')

In [26]:
from tqdm.auto import tqdm

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [28]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|█████████████████████████████| 948/948 [00:21<00:00, 43.11it/s]


In [29]:
quiery = "How do I execute a command in a running docker container?"

In [121]:
def elastic_search(quiery):

    search_quiery= {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": quiery,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                       "course": "machine-learning-zoomcamp"
                   }
            }
        }
    }
    }
    response = es_client.search(index=index_name, body=search_quiery)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    return result_docs

In [122]:
elastic_search(quiery)

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

In [123]:
def build_prompt(quiery, search_results):
    
    prompt_template = """
            
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
            
QUESTION: {question}
        
CONTEXT:
{context}
""".strip()
            

    context = """
    
Q: {question}
A: {text}
""".strip()

    for doc in search_results:
        
        context = context + f"question:{doc['question']}\nanswer:{doc['text']}\n\n"
    
        prompt = prompt_template.format(question=quiery, context=context).strip()
    

    
    return prompt

In [124]:
search_results = elastic_search(quiery)

In [126]:
prompt = build_prompt(quiery, search_results)
len(prompt)

1535

In [131]:
def llm(prompt):
    
    response = client.chat.completions.create(
        
    model='gpt-4o',
        
    messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [132]:
def rag(quiery):
    search_resuls = elastic_search(quiery)
    prompt = build_prompt(quiery, search_results)
    answer = llm(prompt)
    return answer

In [133]:
rag(quiery)

'To execute a command in a running Docker container, you can use the `docker exec` command. Here is the process:\n\n1. First, identify the running container by listing all running containers with:\n   ```shell\n   docker ps\n   ```\n   This will provide you the container IDs.\n\n2. Then, execute a command in the specific container using the container ID:\n   ```shell\n   docker exec -it <container-id> <command>\n   ```\n   For example, to start a bash session, you would use:\n   ```shell\n   docker exec -it <container-id> bash\n   ```\n\nThis approach allows you to run any command inside the context of an already running container.\n\n(Marcos MJD)'

In [135]:
import tiktoken

In [139]:
model_identifier = "gpt-4o"

encoding = tiktoken.encoding_for_model(model_identifier)

# Encode the text
encoded_prompt = encoding.encode(prompt)

# Print the encoded prompt
print(encoded_prompt)

# Print the encoded prompt length
print(len(encoded_prompt))

[63842, 261, 4165, 14029, 29186, 13, 30985, 290, 150339, 4122, 402, 290, 31810, 8099, 591, 290, 40251, 7862, 558, 8470, 1606, 290, 19719, 591, 290, 31810, 8099, 1261, 55959, 290, 150339, 558, 7048, 107036, 25, 3253, 621, 357, 15792, 261, 6348, 306, 261, 6788, 62275, 9282, 3901, 3564, 10637, 50738, 734, 48, 25, 354, 14921, 739, 32, 25, 354, 919, 92, 14921, 25, 5299, 621, 357, 15199, 261, 62275, 9282, 3901, 17021, 25, 35423, 290, 9282, 3621, 306, 25383, 6766, 326, 151187, 290, 7251, 4859, 11, 813, 484, 480, 13217, 261, 38615, 6348, 558, 68923, 2461, 533, 278, 2230, 7962, 4859, 38615, 464, 3365, 523, 3335, 290, 9282, 382, 4279, 6788, 11, 15792, 261, 6348, 306, 290, 4857, 9282, 734, 68923, 10942, 350, 6555, 290, 9282, 26240, 446, 68923, 25398, 533, 278, 464, 6896, 26240, 29, 38615, 198, 6103, 277, 10732, 391, 79771, 1029, 14921, 25, 5299, 621, 357, 5150, 6291, 591, 922, 2698, 7342, 316, 62275, 9282, 3901, 17021, 25, 3575, 665, 5150, 6291, 591, 634, 2698, 7342, 1511, 261, 91238, 9282, 2360,

In [140]:
# Decode the token IDs back to text (to verify)
decoded_text = encoding.decode_single_token_bytes(63842)
print("Decoded text:", decoded_text)

Decoded text: b"You're"
